In [ ]:
from Utils import *
import cv2
from pathlib import Path

In [ ]:
face_image_path = "data/images/StockPhoto3.jpg"
class_indices = [2, 3]
face_segmented = segment_face_et_al(og_img_path=face_image_path, class_indices=class_indices)

In [ ]:
hls_face = cv2.cvtColor(face_segmented, cv2.COLOR_BGR2HLS)
hue = hls_face[:, :, 0]

In [ ]:
# TODO: sort out function in colourspace_stuff
create_colour_heatmap(image_hue=hue, targ_hue=13.37, display=True)
